<a href="https://colab.research.google.com/github/yerinsally/BDA/blob/main/%EC%A0%95%EC%98%88%EB%A6%B0_0402_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

## 교차검증 
- train/test로 나누면 과적합이 발생할 확률이 높다. (overfitting)
- 모델을 학습시에 학습 데이터에 너무 과도하게 최적화 된다.-> 실제로 test 데이터로 수행할 때 예측 성능이 과도하게 떨어진다.
- train 정확도 높고 test 정확도가 낮은 것을 피하기 위해서 교차검증을 진행
- train, test 데이터 외에 검증 valid 데이터를 나눈다.


In [ ]:
fold_iris = load_iris()
features = fold_iris.data
label = fold_iris.target
fold_df_clf = DecisionTreeClassifier()
print(label)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


- 교차검증 진행 : kfold, skfold, cross_val_score, cross_validate
- kfold : train 데이터를 k번 폴드로 나눠서 세트별로 검증 진행

In [ ]:
kfold = KFold(n_splits= 5)
# 모델링을 통해서 결과 값 확인

In [ ]:
kfold.split(features)

<generator object _BaseKFold.split at 0x7f430d945430>

In [ ]:
n_iter = 0 # 5번 진행
cv_accuracy = []
cv_precision = []
cv_recall = []

for train_idx, test_idx in kfold.split(features):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    # 학습 진행
    fold_df_clf.fit(X_train, y_train)
    # 예측값 도출
    fold_pred = fold_df_clf.predict(X_test)
    # 정확도 도출
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, fold_pred), 4)
    precision = precision_score(y_test, fold_pred)
    recall = np.round(recall_score(y_test, fold_pred), 4)
    print('\n{} 교차검증정확도 : {}, 교차검증 precision {}, 교차검증 recall {}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)

mean_accuracy = sum(cv_accuracy) / len(cv_accuracy)
print(mean_accuracy, '평균정확도!')


1 교차검증정확도 : 1.0, 교차검증 precision 0.0, 교차검증 recall 0.0

2 교차검증정확도 : 1.0, 교차검증 precision 1.0, 교차검증 recall 1.0

3 교차검증정확도 : 0.8333, 교차검증 precision 1.0, 교차검증 recall 0.8333

4 교차검증정확도 : 0.9333, 교차검증 precision 0.8333333333333334, 교차검증 recall 1.0

5 교차검증정확도 : 0.7333, 교차검증 precision 0.0, 교차검증 recall 0.0
0.89998 평균정확도!


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- 이진분류가 아니라 다중뷴류임

In [ ]:
# kfold는 클래스에 대한 불균형으로 학습시킨다
kfold_iris_data = load_iris()
kfold_iris_data_df = pd.DataFrame(data=kfold_iris_data.data, columns =kfold_iris_data.feature_names)
kfold_iris_data_df['target']= kfold_iris_data.target
kfold_iris_data_df.target.value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

- kfold는 정답 클래스가 불균형하고 이진 분류가 아닌 경우는 문제가 될 수 있다.
- 교차검증을 진행할 때 내가 예측할 class와 해당 y값의 분포를 꼭 확인해야 한다.

In [ ]:
kfold_re = KFold(n_splits= 3)
kfold_re.split(kfold_iris_data_df)

<generator object _BaseKFold.split at 0x7f430d931c80>

In [ ]:
n_iter = 0

for train_idx, test_idx in kfold.split(kfold_iris_data_df):
    n_iter+=1
    train =kfold_iris_data_df['target'].iloc[train_idx]
    test = kfold_iris_data_df['target'].iloc[test_idx]
    print('교차검증 수 :{}'.format(n_iter))
    print(train.value_counts())
    print(test.value_counts())

교차검증 수 :1
1    50
2    50
0    20
Name: target, dtype: int64
0    30
Name: target, dtype: int64
교차검증 수 :2
2    50
1    40
0    30
Name: target, dtype: int64
0    20
1    10
Name: target, dtype: int64
교차검증 수 :3
0    50
2    50
1    20
Name: target, dtype: int64
1    30
Name: target, dtype: int64
교차검증 수 :4
0    50
1    40
2    30
Name: target, dtype: int64
2    20
1    10
Name: target, dtype: int64
교차검증 수 :5
0    50
1    50
2    20
Name: target, dtype: int64
2    30
Name: target, dtype: int64


- 클래스의 불균형으로 인해서 kfold는 위험할 수 있다.
- 분류문제에서 다중으로 분류하는 경우와 클래스의 분포가 불균형인 경우도 kfold 사용하면 위험하다.
- 회귀문제 예측하는 문제들은 상관없다. 

### Stratified Kfold
- 다중 분류와 클래스에 대한 불균형을 해결하기 위해서 사용

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf_iris= StratifiedKFold(n_splits=3)

In [ ]:
n_iter =0
for train_idx, test_idx in skf_iris.split(kfold_iris_data_df, kfold_iris_data_df['target']): # x값과 y값을 같이 넣어야 한다
    n_iter+=1
    train =kfold_iris_data_df['target'].iloc[train_idx]
    test = kfold_iris_data_df['target'].iloc[test_idx]
    print('교차검증 수 :{}'.format(n_iter))
    print(train.value_counts())
    print(test.value_counts())

교차검증 수 :1
2    34
0    33
1    33
Name: target, dtype: int64
0    17
1    17
2    16
Name: target, dtype: int64
교차검증 수 :2
1    34
0    33
2    33
Name: target, dtype: int64
0    17
2    17
1    16
Name: target, dtype: int64
교차검증 수 :3
0    34
1    33
2    33
Name: target, dtype: int64
1    17
2    17
0    16
Name: target, dtype: int64


- 평가지표 : accuracy precison, recall, f1-score, AUC 
- 평가지표는 대부분 디폴트가 이진 분류일 때 작동한다.
- 예측값과 실제 값을 기반으로 T, F

In [ ]:
skf_iris= StratifiedKFold(n_splits=3)
fold_df_clf = DecisionTreeClassifier(random_state=111)
cv_accuracy = []
cv_precision = []
cv_recall = []

n_iter = 0 # 5번 진행
for train_idx, test_idx in skf_iris.split(features, label):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    # 학습 진행
    fold_df_clf.fit(X_train, y_train)
    # 예측값 도출
    fold_pred = fold_df_clf.predict(X_test)
    # 점수 도출
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, fold_pred),4)
    precision = np.round(precision_score(y_test, fold_pred, average='macro'),4)
    recall = np.round(recall_score(y_test, fold_pred,average='macro'),4)
    print('\n{} 교차검증정확도 : {}, 교차검증 precision {}, 교차검증 recall {}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print(np.mean(accuracy),'평균정확도!') 


1 교차검증정확도 : 0.98, 교차검증 precision 0.9815, 교차검증 recall 0.9792

2 교차검증정확도 : 0.94, 교차검증 precision 0.9407, 교차검증 recall 0.9387

3 교차검증정확도 : 0.98, 교차검증 precision 0.9815, 교차검증 recall 0.9804
0.98 평균정확도!


만약 오류 뜨면 Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

- macro 지정 해줘야 함 : 평가하는 지표 방식을 선언


## cross_val_score

skf, kf 상관없이 cross_val_score는 자동으로 다중분류인지 확인하고 처리해 줌
1. 모델을 입력하는 순간 cross_val 분류인지 회귀인지 자동으로 확인해 준다.
2. x, y값 넣으면 자동으로 클래스 불균형 확인해서 kf, skf 자동으로도 바꿔준다.

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
fold_df_clf = DecisionTreeClassifier(random_state=111)
scores = cross_val_score(fold_df_clf, features, label, scoring='accuracy',cv=5)
print(np.round(scores,4))

[0.9667 0.9667 0.9    0.9667 1.    ]


In [ ]:
scores = cross_val_score(fold_df_clf, features, label, scoring='recall_macro',cv=5) # macro 지정!
print(np.round(scores,4))

[0.9667 0.9667 0.9    0.9667 1.    ]
